In [1]:
import pandas as pd
from tqdm import tqdm
from database.strategy import Strategy
from database.market import Market
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math
import numpy as np

In [2]:
strat_db = Strategy("unity")
market = Market()

In [3]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [ ]:
strat_db.connect()
sim = strat_db.retrieve_data("csim")
strat_db.close()

In [ ]:
availability = []
start = datetime(2018,6,1)
end = datetime(2018,6,1)
for ticker in list(sp5["Symbol"]):
    ticker = ticker.replace(".","-")
    td = sim[(sim["ticker"]==ticker) & (sim["date"]<=end)]
    try:
        availability.append({"ticker":ticker
                             ,"end_date":end <= td.iloc[len(td)-1]["date"]
                             ,"days":(td.iloc[len(td)-1]["date"]- td.iloc[0]["date"]).days
                            ,"records":len(td[td["date"] <= end])
                            ,"pct":math.ceil(len(td[td["date"] <= end])/(end - td.iloc[0]["date"]).days * 100)})
    except Exception as e:
        continue
stuff = pd.DataFrame(availability)
sp5_tickers = list(stuff[(stuff["pct"] >= stuff["pct"].mode().item() - 1) | (stuff["pct"] <= stuff["pct"].mode().item() + 1)]["ticker"].values)

In [ ]:
sim["week"] = [x.week for x in sim["date"]]
sim["year"] = [x.year for x in sim["date"]]

In [ ]:
wgs = sim.groupby(["year","week","ticker"]).mean().reset_index()

In [ ]:
ticker_categories_weeklys = []
for ticker in wgs["ticker"].unique():
    ticker_wgs = wgs[wgs["ticker"] == ticker]
    ticker_wgs["actualclose"] = (ticker_wgs["adjclose"].shift(-1))
    ticker_wgs = ticker_wgs[:-1]
    ticker_wgs["actual_delta"] = (ticker_wgs["actualclose"] - ticker_wgs["adjclose"]) / ticker_wgs["adjclose"]
#     ticker_wgs["sync"] = [0 if row[1]["actual_delta"] * row[1]["weekly_price_regression_prediction_delta"] >= 0 else 1 for row in ticker_wgs.iterrows()]
#     ticker_wgs["category"] = ticker_wgs["sync"].rolling(window=5).mean()
#     ticker_wgs = ticker_wgs[5:]
#     ticker_wgs["category"] = [round(x) for x in ticker_wgs["category"]]
    ticker_wgs["rolling_performance"] = ticker_wgs["actual_delta"].rolling(window=14).mean()
    ticker_categories_weeklys.append(ticker_wgs)
tcw = pd.concat(ticker_categories_weeklys)

In [ ]:
final_sim = sim.merge(tcw[["year","week","ticker"
#                            ,"category"
                           ,"rolling_performance"]],on=["year","week","ticker"],how="left").dropna()
final_sim.drop(["weekly_btc_classification_score","weekly_btc_regression_score"],axis=1,inplace=True)
# final_sim["predicted_delta"] = [-1 * row[1]["weekly_price_regression_prediction_delta"] \
#                           if row[1]["category"] == 1 else row[1]["weekly_price_regression_prediction_delta"] \
#                           for row in final_sim.iterrows()]

In [ ]:
final_sim.sort_values("date")[["high","low","adjclose"]].head(20)

In [ ]:
def backtest(sim,trend,classifier,score,weekly_delta_req,ascending,seats,performance_filter):
    trades = []
    sim_columns = sim.columns
    sorted_column = "weekly_price_regression_prediction_delta"
    sc = [x for x in sim_columns if "score" in x]
    rc = [x for x in sim_columns if "regression_prediction" in x]
    blacklist = pd.DataFrame([{"ticker":"ZZZZZ","start":datetime(2016,4,1),"end":datetime(2016,4,14)}])
    for i in range(1,seats):
        date = start
        while date <= end:
            if date.weekday() > 4:
                date = date + timedelta(days=7-date.weekday())
            blacklist_tickers = blacklist[(blacklist["start"] <= date) & (blacklist["end"] >= date)]["ticker"]
            taken = []
            for col in sc:
                sim = sim[sim[col] >= score]
            delta_column = "weekly_price_regression_prediction_delta"
            todays_sim = sim[(~sim["ticker"].isin(blacklist_tickers)) & (sim["ticker"].isin(sp5_tickers))
                            & (sim["date"] == date)
                             & (sim[delta_column] >= weekly_delta_req)
                             & (sim[delta_column] <= 1)]
            if performance_filter:
                todays_sim = todays_sim.sort_values("rolling_performance",ascending=False) \
                                .head(int(len(todays_sim["ticker"].unique())/2))
            if todays_sim.index.size >= 1:
                offerings = todays_sim.sort_values(sorted_column,ascending=ascending)
                offerings = offerings[(offerings["{}_classification_prediction".format(classifier)] == trend)]
#                 offerings = pd.concat([offerings[(offerings["category"]==0) \
#                                                  & (offerings["{}_classification_prediction".format(classifier)] == 1)], \
#                                        offerings[(offerings["category"]==1) \
#                                                  & (offerings["{}_classification_prediction".format(classifier)] == 0)]])
                if offerings.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for offering in range(offerings.index.size):
                        try:
                            trade_ticker = offerings.iloc[offering]["ticker"]
                            trade = sim[(sim["ticker"] == trade_ticker) & (sim["date"] > date)].iloc[0]                     
                            sell_date = trade["date"] + timedelta(days=1)
                            sell_trades = sim[(sim["date"] >= sell_date)  & (sim["date"] <= sell_date + timedelta(days=4)) & (sim["ticker"] == trade["ticker"])]
                            if sell_trades.index.size < 1:
                                if offering == offerings.index.size - 1:
                                    date = date + timedelta(days=1)
                                    break
                                else:
                                    continue
                            else:
                                sell_trades["delta"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                sell_trade = sell_trades.sort_values("delta",ascending=False).iloc[0]
                                trade["sell_price"] = sell_trade["adjclose"]
                                trade["sell_date"] = sell_trade["date"]
                                trade["sell_delta"] = float(sell_trade["delta"])
                                trade["seat"] = i
#                                 trade["category"] = sell_trade["category"]
                                trade["predicted_delta"] = sell_trade[delta_column]
                                blacklist = blacklist.append([{"ticker":trade["ticker"],"start":date,"end":trade["sell_date"]}])
                                trades.append(trade)
                                date = sell_trade["date"] + timedelta(days=1)
                                break
                        except Exception as e:
                            print("packaging",i,str(e))
                            print("sell_trades",trade_ticker, sell_trades.index.size)
                            date = date + timedelta(days=1)
            else:
                date = date + timedelta(days=1)
    return trades

In [ ]:
epoch = 0
seats = 11
score_range = range(70,90,10)
wdr_range = range(0,15,5)
ascending_list = [True,False]
categories = [True,False]
start = datetime(2016,6,1)
end = datetime(2020,6,1)
classifiers = ["consolidated","trade_signal"]
pf_list =  [True,False]
strat_db.connect()
strat_db.drop_table("tsmc_epochs")
for i in range(len(score_range)*len(wdr_range)*len(ascending_list) *len(pf_list) * len(sp5["GICS Sector"].unique())):
    strat_db.drop_table("tsmc_{}".format(i))
for trend in [0,1]:
    for classifier in classifiers:
        for pf in tqdm(pf_list):
            for s in tqdm(score_range):
                score = s/100
                for wdr in wdr_range:
                    for ascending in ascending_list:
                        weekly_delta_req = wdr/100
                        epoch_dict = {"epoch":epoch,"trend":trend,"classifier":classifier,"score":score,"ascending":ascending,"wdr":wdr,"pf":pf}
                        ts = backtest(final_sim.copy(),trend,classifier,score,weekly_delta_req,ascending,seats,pf)
                        if len(ts) > 0:
                            strat_db.store_data("tsmc_epochs",pd.DataFrame([epoch_dict]))
                            strat_db.store_data("tsmc_{}".format(epoch),pd.DataFrame(ts))
                        epoch += 1
strat_db.close()